In [1]:
%load_ext line_profiler

In [2]:
import cv2
import numpy as np
import knn
import inpainting_functions

import random

In [3]:
DEFINED = 0
UNDEFINED = 255

In [4]:
def create_pattern(mask: np.ndarray, origin: tuple, k: int) -> list[tuple]:
    origin_y, origin_x = origin
    neighbors = knn.nn_circular_native(mask, origin, k)
    neighbors_relative = []
    for y, x in neighbors:
        neighbors_relative.append((y - origin_y, x - origin_x))
    return neighbors_relative


In [5]:
def generate_candidates(mask: np.ndarray, neighbors_relative: list[tuple], n: int) -> list[tuple]:
    height, width = mask.shape
    candidates = []
    while len(candidates) < n:
        y, x = np.random.randint(0, height), np.random.randint(0, width)
        if mask[y, x] == UNDEFINED:
            continue
        for y_n, x_n in neighbors_relative:
            if y + y_n < 0 or y + y_n >= height or x + x_n < 0 or x + x_n >= width or mask[y + y_n, x + x_n] == UNDEFINED:
                break
        else:
            candidates.append((y, x))
    return candidates

In [6]:
def sparse_l2(image: np.ndarray, origin: tuple, candidate: tuple, neighbors: list[tuple]) -> float:
    origin_y, origin_x = origin
    candidate_y, candidate_x = candidate
    distance = 0
    for neighbor_y, neighbor_x in neighbors:
        distance += np.sum(np.power(image[origin_y + neighbor_y, origin_x + neighbor_x] - image[candidate_y + neighbor_y, candidate_x + neighbor_x], 2))

    return np.sqrt(distance)

In [7]:
def choose_candidate(image: np.ndarray, origin: tuple, candidates: list[tuple], neighbors: list[tuple]) -> tuple:
    distances = []
    for candidate in candidates:
        distances.append(sparse_l2(image, origin, candidate, neighbors))

    return candidates[np.argmin(distances)]

In [8]:
def choose_candidate_native_l2(image: np.ndarray, origin: tuple, candidates: list[tuple], neighbors: list[tuple]) -> tuple:
    distances = []
    for candidate in candidates:
        distances.append(inpainting_functions.sparse_l2(image, origin, candidate, neighbors))
    return candidates[np.argmin(distances)]

In [9]:
def inpainting(image: np.ndarray, mask: np.ndarray, n_neighbors: int = 10, n_candidates: int = 100):
    image[mask == UNDEFINED] = UNDEFINED
    undefined_pixels = np.argwhere(mask == UNDEFINED).tolist()
    iteration = 0
    while undefined_pixels:
        origin = random.choice(undefined_pixels)
        pattern = create_pattern(mask, origin, n_neighbors)
        candidates = generate_candidates(mask, pattern, n_candidates)
        candidate = choose_candidate(image, origin, candidates, pattern)
        image[origin[0], origin[1]] = image[candidate[0], candidate[1]]
        undefined_pixels.remove(origin)

        if iteration == 100:
            break
        if iteration % 100 == 0:
            cv2.imwrite(f"/tmp/inpainting/iteration{iteration:04}.png", image)
        iteration += 1
    cv2.imwrite(f"/tmp/inpainting/iteration{iteration:04}.png", image)

In [10]:
image01 = cv2.imread("data/image_01.jpg")
mask01 = cv2.imread("data/mask_01.png", cv2.IMREAD_GRAYSCALE)

%lprun -f inpainting inpainting(image01, mask01)

Timer unit: 1e-09 s

Total time: 0.972652 s
File: /tmp/ipykernel_12869/2782660982.py
Function: inpainting at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def inpainting(image: np.ndarray, mask: np.ndarray, n_neighbors: int = 10, n_candidates: int = 100):
     2         1     184846.0 184846.0      0.0      image[mask == UNDEFINED] = UNDEFINED
     3         1    1368621.0    1e+06      0.1      undefined_pixels = np.argwhere(mask == UNDEFINED).tolist()
     4         1        230.0    230.0      0.0      iteration = 0
     5       101      26427.0    261.7      0.0      while undefined_pixels:
     6       101     304949.0   3019.3      0.0          origin = random.choice(undefined_pixels)
     7       101    6510957.0  64464.9      0.7          pattern = create_pattern(mask, origin, n_neighbors)
     8       101  231048319.0    2e+06     23.8          candidates = generate_candidates(mask, pattern, n_candidates

In [11]:
def inpainting_2(image: np.ndarray, mask: np.ndarray, n_neighbors: int = 10, n_candidates: int = 100):
    image[mask == UNDEFINED] = UNDEFINED
    undefined_pixels = np.argwhere(mask == UNDEFINED).tolist()
    iteration = 0
    while undefined_pixels:
        origin = random.choice(undefined_pixels)
        pattern = create_pattern(mask, origin, n_neighbors)
        candidates = generate_candidates(mask, pattern, n_candidates)
        candidate = choose_candidate_native_l2(image, origin, candidates, pattern)
        image[origin[0], origin[1]] = image[candidate[0], candidate[1]]
        undefined_pixels.remove(origin)

        if iteration == 100:
            break
        if iteration % 100 == 0:
            cv2.imwrite(f"/tmp/inpainting/iteration{iteration:04}.png", image)
        iteration += 1
    cv2.imwrite(f"/tmp/inpainting/iteration{iteration:04}.png", image)

image01 = cv2.imread("data/image_01.jpg")
mask01 = cv2.imread("data/mask_01.png", cv2.IMREAD_GRAYSCALE)
%lprun -f inpainting_2 inpainting_2(image01, mask01)

Timer unit: 1e-09 s

Total time: 0.266998 s
File: /tmp/ipykernel_12869/2995356549.py
Function: inpainting_2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def inpainting_2(image: np.ndarray, mask: np.ndarray, n_neighbors: int = 10, n_candidates: int = 100):
     2         1     158897.0 158897.0      0.1      image[mask == UNDEFINED] = UNDEFINED
     3         1     477503.0 477503.0      0.2      undefined_pixels = np.argwhere(mask == UNDEFINED).tolist()
     4         1        230.0    230.0      0.0      iteration = 0
     5       101      22552.0    223.3      0.0      while undefined_pixels:
     6       101     246647.0   2442.0      0.1          origin = random.choice(undefined_pixels)
     7       101    5236624.0  51847.8      2.0          pattern = create_pattern(mask, origin, n_neighbors)
     8       101  232082625.0    2e+06     86.9          candidates = generate_candidates(mask, pattern, n_candid

In [16]:
def inpainting_3(image: np.ndarray, mask: np.ndarray, n_neighbors: int = 10, n_candidates: int = 100):
    image[mask == UNDEFINED] = UNDEFINED
    undefined_pixels = np.argwhere(mask == UNDEFINED).tolist()
    iteration = 0
    while undefined_pixels:
        origin = random.choice(undefined_pixels)
        pattern = create_pattern(mask, origin, n_neighbors)
        candidates = inpainting_functions.generate_candidates(mask, pattern, n_candidates)
        candidate = choose_candidate_native_l2(image, origin, candidates, pattern)
        image[origin[0], origin[1]] = image[candidate[0], candidate[1]]
        undefined_pixels.remove(origin)

        if iteration == 100:
            break
        if iteration % 100 == 0:
            cv2.imwrite(f"/tmp/inpainting/iteration{iteration:04}.png", image)
        iteration += 1
    cv2.imwrite(f"/tmp/inpainting/iteration{iteration:04}.png", image)

image01 = cv2.imread("data/image_01.jpg")
mask01 = cv2.imread("data/mask_01.png", cv2.IMREAD_GRAYSCALE)
%lprun -f inpainting_3 inpainting_3(image01, mask01)

Timer unit: 1e-09 s

Total time: 0.0510332 s
File: /tmp/ipykernel_12869/1648953004.py
Function: inpainting_3 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def inpainting_3(image: np.ndarray, mask: np.ndarray, n_neighbors: int = 10, n_candidates: int = 100):
     2         1     391863.0 391863.0      0.8      image[mask == UNDEFINED] = UNDEFINED
     3         1    1166412.0    1e+06      2.3      undefined_pixels = np.argwhere(mask == UNDEFINED).tolist()
     4         1        511.0    511.0      0.0      iteration = 0
     5       101      30616.0    303.1      0.1      while undefined_pixels:
     6       101     296537.0   2936.0      0.6          origin = random.choice(undefined_pixels)
     7       101    7086233.0  70160.7     13.9          pattern = create_pattern(mask, origin, n_neighbors)
     8       101    1599453.0  15836.2      3.1          candidates = inpainting_functions.generate_candidates(m

In [22]:
def inpainting_4(image: np.ndarray, mask: np.ndarray, n_neighbors: int = 10, n_candidates: int = 100):
    image[mask == UNDEFINED] = UNDEFINED
    undefined_pixels = np.argwhere(mask == UNDEFINED).tolist()
    iteration = 0
    while undefined_pixels:
        origin = random.choice(undefined_pixels)
        pattern = create_pattern(mask, origin, n_neighbors)
        candidates = inpainting_functions.generate_candidates(mask, pattern, n_candidates)
        candidate = choose_candidate_native_l2(image, origin, candidates, pattern)
        image[origin[0], origin[1]] = image[candidate[0], candidate[1]]
        undefined_pixels.remove(origin)

        if iteration % 100 == 0:
            cv2.imwrite(f"/tmp/inpainting/iteration{iteration:04}.png", image)
        iteration += 1
    cv2.imwrite(f"/tmp/inpainting/iteration{iteration:04}.png", image)

image = cv2.imread("data/1d.jpg")
mask = cv2.imread("data/mask_1.png", cv2.IMREAD_GRAYSCALE)
inpainting_4(image, mask)

(34, 188)
(267, 384)
(278, 111)
(75, 376)
(42, 205)
(252, 87)
(15, 109)
(180, 47)
(359, 203)
(127, 302)
(280, 156)
(9, 297)
(45, 116)
(135, 94)
(321, 378)
(296, 206)
(313, 375)
(26, 122)
(17, 179)
(239, 373)
(261, 258)
(32, 222)
(319, 0)
(107, 330)
(249, 351)
(287, 282)
(11, 375)
(308, 169)
(284, 283)
(277, 269)
(272, 346)
(340, 208)
(25, 283)
(309, 95)
(200, 362)
(331, 252)
(251, 99)
(328, 88)
(355, 230)
(283, 327)
(74, 119)
(313, 269)
(327, 248)
(48, 193)
(148, 339)
(105, 290)
(372, 53)
(355, 250)
(299, 348)
(74, 238)
(4, 314)
(26, 99)
(317, 61)
(9, 332)
(69, 302)
(152, 314)
(149, 359)
(47, 305)
(225, 78)
(315, 158)
(78, 328)
(145, 309)
(101, 322)
(129, 372)
(270, 331)
(323, 77)
(129, 315)
(367, 344)
(282, 20)
(183, 76)
(60, 303)
(326, 346)
(52, 91)
(281, 249)
(54, 280)
(285, 280)
(167, 344)
(322, 256)
(268, 324)
(216, 343)
(270, 84)
(324, 356)
(266, 327)
(75, 284)
(134, 306)
(270, 117)
(16, 274)
(304, 66)
(6, 331)
(126, 306)
(273, 108)
(49, 380)
(57, 332)
(263, 348)
(37, 244)
(343, 

IndexError: index 382 is out of bounds for axis 0 with size 382